# Training notebook

Here we tune parameters of models. From the 7 available seasons, we use first 4 for training and the rest 3 go in testing. There is no fitting on data as such and so we just take the average performance over 4 training seasons to see which parameters work best.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
np.random.seed(1234)

import sys
sys.path.append("../src")
from utils import data as udata
from utils import dists as udists
from utils import misc as u
from truth import mask_truths
from predictors import make_predictor
import losses
from pymmwr import Epiweek
from tqdm import tqdm, trange
import ledge.merge as merge
import ledge.update as update
from functools import partial

In [2]:
EXP_DIR = "../data/processed/cdc-flusight-ensemble/"
OUTPUT_DIR = "../models/cdc-flusight-ensemble/"
TARGET = "1-ahead"
MAX_LAG = 29
# REGIONS = ["nat", *[f"hhs{i}" for i in range(1, 11)]]
REGIONS = ["nat"] # Speed up!
TRAINING_SEASONS = list(range(2010, 2014))
LOSS_FN = losses.ploss

In [3]:
components = [udata.Component(EXP_DIR, m) for m in udata.available_models(EXP_DIR)]
ad = udata.ActualData(EXP_DIR)

# Evaluation

In [36]:
def evaluate(predictor, loss_fn=losses.ploss):
    """
    Evaluate the predictor over all training seasons and regions and return mean score
    """
    
    first_losses = []
    final_losses = []
    
    with tqdm(total=len(TRAINING_SEASONS) * len(REGIONS)) as pbar:
        for season in TRAINING_SEASONS:
            for region in REGIONS:
                truths = [ad.get(TARGET, region, season, lag=l) for l in range(MAX_LAG + 1)]
                c_preds = [cmp.get(TARGET, region, season) for cmp in components]
            
                first_truth = merge.earliest(truths)
                final_truth = merge.latest(truths)
                pred = predictor(truths, c_preds)
                first_losses.append(float(loss_fn(pred, first_truth).mean()))
                final_losses.append(float(loss_fn(pred, final_truth).mean()))
                pbar.update()

    return {
        "first_loss": np.mean(first_losses),
        "final_loss": np.mean(final_losses)
    }

# Tuning

In [39]:
# Merge strategy
MERGE_FN = merge.latest

## Follow the leader

In [40]:
losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update.ftl), LOSS_FN)
print(f"{losses['first_loss'], losses['final_loss']}")

100%|██████████| 4/4 [00:19<00:00,  4.88s/it]

(0.8558966113027653, 0.8934405764400863)


## MW

In [41]:
for eta in np.linspace(0.5, 1.0, 11):
    update_fn = partial(update.mw, eta=eta)
    losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
    print(f"{eta}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

0.5: (0.8839089347666891, 0.8973592565817979)


  0%|          | 0/4 [00:00<?, ?it/s]

0.55: (0.8819961697712663, 0.8963151587345148)


  0%|          | 0/4 [00:00<?, ?it/s]

...

0.9: (0.8672534561248718, 0.8901393386791897)


  0%|          | 0/4 [00:00<?, ?it/s]

0.95: (0.8662375708334357, 0.8903008346009428)


100%|██████████| 4/4 [00:21<00:00,  5.29s/it]

1.0: (0.8697122091091656, 0.8942047744704603)


## Hedge

In [42]:
for eta in np.linspace(0.5, 5.0, 11):
    update_fn = partial(update.hedge, eta=eta)
    losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
    print(f"{eta}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

0.5: (0.8892561550495424, 0.9007019452019019)


  0%|          | 0/4 [00:00<?, ?it/s]

0.95: (0.882868569951269, 0.8970867948698384)


  0%|          | 0/4 [00:00<?, ?it/s]

...

4.1: (0.864476030473778, 0.8911387987320996)


  0%|          | 0/4 [00:00<?, ?it/s]

4.55: (0.8636649135021445, 0.8909539240942543)


100%|██████████| 4/4 [00:22<00:00,  5.64s/it]

5.0: (0.86302364913953, 0.890812073202538)


## Fixed share

In [45]:
for alpha in np.linspace(0, 1, 11):
    for eta in np.linspace(0.5, 5.0, 11):
        update_fn = partial(update.fixed_share, eta=eta, alpha=alpha)
        losses = evaluate(make_predictor(LOSS_FN, MERGE_FN, update_fn), LOSS_FN)
        print(f"{eta}, {alpha}: {losses['first_loss'], losses['final_loss']}")

  0%|          | 0/4 [00:00<?, ?it/s]

0.5, 0.0: (0.8892561550495424, 0.9007019452019019)


  0%|          | 0/4 [00:00<?, ?it/s]

0.95, 0.0: (0.882868569951269, 0.8970867948698384)


  0%|          | 0/4 [00:00<?, ?it/s]

...

4.1, 1.0: (0.8994796651646969, 0.907850401024811)


  0%|          | 0/4 [00:00<?, ?it/s]

4.55, 1.0: (0.8994796651646969, 0.9078504010248108)


100%|██████████| 4/4 [00:48<00:00, 12.01s/it]

5.0, 1.0: (0.8994796651646968, 0.9078504010248108)
